<!-- PROJECT LOGO -->
<br />
<div align="center">
  <a href="https://www.javeriana.edu.co/inicio">
    <img src="https://www.camaradelpacifico.org/tl_files/Casos%20Exito/AFILIADOS/JAVERIANA/tiara_javeriana.png" alt="Logo" width="300" height="100">
  </a>

  <h2 align="center"> Proyecto 1: Data Pipeline</h2>
  <h2 align="center"> Topicos Avanzados de IA</h2>
  <h4 align="center"> Yefry Moncada Linares, David Andrés Peña</h4>
    
***

  <p align="center">
    El siguiente notebook se realizará el proceso detallado de evaluación, ingesta, validación y transformación de datos para el conjunto de datos **Tipo de Cubierta Forestal**</p>
</div>


* [1. Selección de características](#topic1)
* [2. Ingesta del Conjunto de datos](#topic2)
* [3. Generación de las estadísticas del conjunto de datos](#topic3)
* [4. Creación de un esquema según el conocimiento del dominio.](#topic4)
* [5. Creación de entornos de esquema](#topic5)
* [6. Visualización de las anomalías del conjunto de datos](#topic6)
* [7. Preprocesamiento, transformación e ingeniería de sus características](#topic7)
* [8. Seguimiento de la procedencia de su flujo de datos utilizando metadatos de ML](#topic8)

### Importamos las librerias

In [331]:
import os
import requests
import tensorflow as tf
import pandas as pd

import tensorflow_data_validation as tfdv
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.v1.components import ImportSchemaGen
from tfx.types import standard_artifacts

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tensorflow_metadata.proto.v0 import schema_pb2
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

### Cargamos el Dataset

In [ ]:
#Opcional eliminamos el dataset para reproducir correctamente todos lo pasos del notebook
!rm -r -f data/covertype/covertype_train.csv

In [260]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)


## 3. Seleccion de características

Reducimos la cantidad de caracteristicas para alimentar el modelo, ayudando a reducir la complejidad y el ahorro de recursos.

- Univariate-feature-selection
- SelectKbest


In [261]:
ORIGINAL_DATASET_PATH = 'data/covertype/covertype_train.csv'

In [262]:
df = pd.read_csv(ORIGINAL_DATASET_PATH)
print(df.shape)
df.head()

(116203, 13)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [263]:
df.dtypes

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object

In [264]:
df_numerical = df.select_dtypes(include='int')
print(df_numerical.shape)
df_numerical.head()

(116203, 11)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,1
1,2876,3,18,485,71,2495,192,202,144,1557,1
2,3171,315,2,277,9,4374,213,237,162,1052,0
3,3087,342,13,190,31,4774,193,221,166,752,0
4,2835,158,10,212,41,3596,231,242,141,3280,1


Al observar los tipos de datos de cada columna y la descripción del conjunto de datos, se puede evidenciar que la mayoria son funciones numéricas y solo dos no lo son **"Wilderness_Area y Soil_Type"**, sin embargo cabe destacar que nuestra variable objetivo **"Cover_type"** tambien es una variable con rasgos categórico.

In [265]:
df_categorical = df.select_dtypes(include='object')
print(df_categorical.shape)
df_categorical.head()

(116203, 2)


,Wilderness_Area,Soil_Type
0,Commanche,C7202
1,Commanche,C7757
2,Rawah,C7745
3,Rawah,C7745
4,Rawah,C4744


### Skelearn selección de caracteristicas

In [266]:
X = df_numerical.drop('Cover_Type', axis=1)
print(f'1. El tamaño del dataframe de caracteristicas númericas es {X.shape}') 
y = df_numerical['Cover_Type']
print(f'2. El tamaño de la variable objetivo es{y.shape}') 

1. El tamaño del dataframe de caracteristicas númericas es (116203, 10)
2. El tamaño de la variable objetivo es(116203,)


Utilizamos el modulo de scikit-learn para realizar la selección de caracteristicas

In [267]:
feature_selection = SelectKBest(f_classif, k=8)
x_new = feature_selection.fit_transform(X, y)

feature_mask = feature_selection.get_support()

In [268]:
X.columns[feature_mask]

Index(['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')

A continuación, podemos observar el resultado de la selección de caracteristicas significativas para el modelo utilizando el univariate-feature-selection, las columnas que no fueron seleccionadas son la de **Aspect** y **Hillshade3pm**

In [269]:
feature_mask

array([ True, False,  True,  True,  True,  True,  True,  True, False,
        True])

In [83]:
# TODO: Crear tabla de la IMAGEN del PDF
# fs = SelectKBest(score_func=f_regression, k=5)
# print(zip(fs.get_support(), features_columns))

In [270]:
# columnas resultantes
X.columns[feature_mask]

Index(['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')

In [271]:
df_cover_type = pd.DataFrame(data = x_new, columns = X.columns[feature_mask]) 
#concatenamos nuevamente la columna CoverType para rehacer el dataframe original del subconjunto seleccionado
df_cover_type['Cover_Type'] = df_numerical['Cover_Type']
df_cover_type.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2991,7,67,11,1015,233,234,1570,1
1,2876,18,485,71,2495,192,202,1557,1
2,3171,2,277,9,4374,213,237,1052,0
3,3087,13,190,31,4774,193,221,752,0
4,2835,10,212,41,3596,231,242,3280,1


Debido a que en pasos posteriores realizaremos la canalización de los datos, guardaremos el subconjunto seleccionado en un archivo CSV.

In [272]:
df_cover_type.to_csv('./data/covertype/covertype_train.csv',index=False)

## 4. Data Pipeline

Con el subconjunto seleccionado de caracteristicas, empezaremos a crear la canalización de los datos tulizando los componentes de TFX.

### 4.1. Configurar el contexto interactivo


Definimos las rutas de los datos y las carpetas donde se iran guardando los metadatos del proceso. Al iniciar el TFX Interactive Context se nos permite crear y ejecutar componentes TFX en un modo interactivo, creando visualizaciones más amigables para el proceso de experimentación.

In [273]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './data/covertype'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

#Inicializamos el contexto interactivo
context = InteractiveContext(pipeline_root=_pipeline_root)

In [274]:
!head {_data_filepath}

Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Cover_Type
2991,7,67,11,1015,233,234,1570,1
2876,18,485,71,2495,192,202,1557,1
3171,2,277,9,4374,213,237,1052,0
3087,13,190,31,4774,193,221,752,0
2835,10,212,41,3596,231,242,3280,1
3185,7,470,68,883,222,225,2187,0
3157,29,361,95,1756,248,179,2766,1
3152,7,0,0,5549,201,235,2100,1
3302,23,309,-30,806,252,212,3537,1


In [275]:
!head {_data_filepath}

Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Cover_Type
2991,7,67,11,1015,233,234,1570,1
2876,18,485,71,2495,192,202,1557,1
3171,2,277,9,4374,213,237,1052,0
3087,13,190,31,4774,193,221,752,0
2835,10,212,41,3596,231,242,3280,1
3185,7,470,68,883,222,225,2187,0
3157,29,361,95,1756,248,179,2766,1
3152,7,0,0,5549,201,235,2100,1
3302,23,309,-30,806,252,212,3537,1


### 4.2. Generando Ejemplos

Ell primer paso para iniciar nuestra canalización de los datos es ingerir los conjuntos de datos de entrenamiento y prueba. 

El componente ExampleGen ingiere datos en una canalización TFX. Consume archivos/servicios externos para generar un conjunto de archivos en el formato TFRecord, que será utilizado por otros componentes TFX. También puede mezclar los datos y dividirlos en un número arbitrario de particiones.

In [276]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=_data_root)

In [278]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 20
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [280]:
example_gen

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7f78dfa6b9d0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7f78dfa6b550>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={})})

In [281]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/20


### 4.3. Estadísticas

El StatisticsGen generara estadísticas de datos que pueden ser utilizadas por otros componentes TFX. StatisticsGen utiliza la validación de datos de TensorFlow. 

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 21
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

En nuestro caso StadisticsGen no interpretó correctamente el campo Cover_Type. Aunque el campo esta codificado como número entero, deben interpretarse como categórico en lugar de numérico en pasos posteriores se corregira esto.

In [283]:
context.show(statistics_gen.outputs['statistics'])

### 4.4. Inferir el esquema

El SchemaGen tambien utiliza Tensorflow data validation para generar un schema basado en las estadisticas que generamos anteriormente.

Al crear el Schema podemor generar nuevas designaciones o dominios a las caracteristicas correctas, para volver a generar las estadisticas y finalizar el esquema.

In [284]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 22
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

El esquema describe las propiedades esperadas de los datos. Algunas de estas propiedades son:

qué características se espera que estén presentes
su tipo
el número de valores para una característica en cada ejemplo
la presencia de cada característica en todos los ejemplos
los dominios de características esperados.
En resumen, el esquema describe las expectativas de datos "correctos" y, por lo tanto, puede usarse para detectar errores en los datos (descritos a continuación). Además, el mismo esquema se puede usar para configurar Tensorflow Transform para transformaciones de datos. Tenga en cuenta que se espera que el esquema sea bastante estático, por ejemplo, varios conjuntos de datos pueden ajustarse al mismo esquema, mientras que las estadísticas (descritas anteriormente) pueden variar según el conjunto de datos.

Dado que escribir un esquema puede ser una tarea tediosa, especialmente para conjuntos de datos con muchas funciones, TFDV proporciona un método para generar una versión inicial del esquema basada en las estadísticas descriptivas:

In [ ]:
# Visualizamos el schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


In [286]:
schema_gen.outputs['schema']

OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={})

### 4.5. Curando el esquema


Ahora con el esquema correctamente inferido, procedemos con la aplicación de las reglas especificas del dataset para limitar su domino y robustez. Para ello utilizaremos TFDV para cargar el schmea el cual es un archivo de protocolo que se ha guardado en la base de metadatos.

Los parametros que van a modificar son los siguientes 

- **Hillshade 9am:** 0 a 255
- **Hillshade Noon:** 0 a 255
- **Slope:** 0 a 90
- **Cover Type:** 0 to 6, tambien caterocia

https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/get_domain

In [288]:
# Get the schema uri
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [289]:
# Restrict the range of the `Hillshade 9am` feature
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(name='age', min=0, max=255))

# Restrict the range of the `Hillshade Noon` feature
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(name='age', min=0, max=255))

# Restrict the range of the `Hillshade Slope` feature
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(name='age', min=0, max=90))

Declaramos adicionalmente **"Cover Type“** como una variable categórica.

A diferencia de las otras cuatro características, los números enteros del 0 al 6 aquí corresponden a una etiqueta designada y no a una medida cuantitativa.

Utilizamos set domain() de TFDV para configurar esto.


In [290]:
# Restrict the range of the `Cover_Type feature
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='age', min=0, max=6, is_categorical=True))

In [291]:
# Display the modified schema. Notice the `Domain` column of `age`.
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


Como se puede evidenciar una vez limitados los dominios de las variables 

### 4.6. Entornos de esquema

In [293]:
# Create schema environments for training and serving
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

Se espera que, la columna que falta este marcada. Para solucionar esto, debe configurar el esquema para detectar cu´ando se usa para capacitaci´on o para inferencia/servicio. Puede hacerlo configurando
entornos de esquema. Al aplicar esta configuraci´on entre entornos debe pasar la validaci´on sin problema y se deber´a guardar este esquema seleccionado en un directorio local para poder importarlo a nuestro pipeline de TFX.


In [294]:
# Omit label from the serving environment
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')

In [295]:
# Declare the path to the updated schema directory
_updated_schema_dir = f'{_pipeline_root}/updated_schema'

# Create the said directory
!mkdir -p {_updated_schema_dir}

# Declare the path to the schema file
schema_file = os.path.join(_updated_schema_dir, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)

Como verificaci´on, debe mostrar el esquema que acaba de guardar y verificar que contiene los cambios que introducidos. Debe mostrar los rangos en la columna Domain y debe haber dos entornos disponibles.

In [296]:
# Get the schema pbtxt file from the SchemaGen output
updated_schema = tfdv.load_schema_text(os.path.join(_updated_schema_dir, 'schema.pbtxt'))

In [297]:
tfdv.display_schema(updated_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


### 4.7. Genere nuevas estadísticas usando el esquema actualizado

#### 4.7.1 Generando nuevas estadisticas con el esquema actualizado mediante TensorFlow Data Validation y el esquema actualizado anteriormente

In [298]:
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

train_stats = tfdv.generate_statistics_from_csv(
    data_location=_data_filepath,
    stats_options=stats_options,
)

tfdv.visualize_statistics(train_stats)

#### 4.7.3 Generando nuevas estadisticas con el esquema actualizado mediante ImportSchemaGen

In [414]:
# Crear el componente ImportSchemaGen
update_schema_gen = ImportSchemaGen(schema_file)

user_schema_importer = Importer(
    source_uri=_updated_schema_dir, # directory containing only schema text proto
    artifact_type=standard_artifacts.Schema)


In [415]:
context.run(update_schema_gen)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 36
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [416]:
# get the artifact object
artifact = update_schema_gen.outputs['schema'].get()[0]
artifact 

.type,<class 'tfx.types.standard_artifacts.Schema'>
.uri,./pipeline/ImportSchemaGen/schema/23


In [417]:
context.show(update_schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


In [418]:
# Crear el artefacto Schema
schema_artifact = standard_artifacts.Schema()

schema_artifact.uri = schema_file

In [419]:
update_schema_gen.outputs['schema']._artifacts[0] = schema_artifact

In [420]:
update_schema_gen.outputs['schema']

OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={})

In [421]:
context.run(update_schema_gen)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 37
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [425]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
update_statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=update_schema_gen.outputs['schema'],)

# Execute the component
context.run(update_statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 39
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [426]:
context.show(update_statistics_gen.outputs['statistics'])

### 4.8. Comprobar anomalías


Ahora debe comprobar si el conjunto de datos tiene alguna anomal´ıa con respecto al esquema.
Puede hacerlo f´acilmente con el componente ExampleValidator. Compruebe si hay alguna anomal´ıa
utilizando ExampleValidator. Deber´a pasar las estad´ısticas y el esquema actualizados de las secciones
anteriores.

In [427]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = ExampleValidator(
    statistics=update_statistics_gen.outputs['statistics'],
    schema=update_schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 40
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [428]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

### 4.9. Ingeniería de características


In [433]:
# Set the constants module filename
_cover_type_constants_module_file = 'cover_type_constants.py'

In [434]:
%%writefile {_cover_type_constants_module_file}

# No se tienen caracterisiticas categoricas debido al subconjunto de puntos anteriores
CATEGORICAL_FEATURE_KEYS = ['Cover_Type']

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = [
    'Elevation', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points',
    'Horizontal_Distance_To_Hydrology','Horizontal_Distance_To_Roadways','Slope',
    'Vertical_Distance_To_Hydrology'
]

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['Slope']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'Slope': 4}

# Feature that the model will predict
LABEL_KEY = 'Cover_Type'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing cover_type_constants.py


### 4.10. Función de preprocesamiento

In [435]:
# Set the transform module filename
_cover_type_transform_module_file = 'cover_type_transform.py'

In [442]:
%%writefile {_cover_type_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_type_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = cover_type_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = cover_type_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = cover_type_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = cover_type_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY = cover_type_constants.LABEL_KEY
_transformed_name = cover_type_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
    
    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs

Overwriting cover_type_transform.py


### 4.11. Transformar

In [443]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=update_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_census_transform_module_file))

# Run the component
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying cover_type_constants.py -> build/lib
copying cover_type_transform.py -> build/lib
installing to /tmp/tmpdnovc8uq
running install
running install_lib
copying build/lib/cover_type_constants.py -> /tmp/tmpdnovc8uq
copying build/lib/cover_type_transform.py -> /tmp/tmpdnovc8uq
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpdnovc8uq/tfx_user_code_Transform-0.0+0544a8596c2fadf49c533574ac47f2f8349efcf9f4e159f1

Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+0544a8596c2fadf49c533574ac47f2f8349efcf9f4e159f15559617240c354e5-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+0544a8596c2fadf49c533574ac47f2f8349efcf9f4e159f15559617240c354e5-py3-none-any.whl


2023-03-11 21:57:55.049096: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-11 21:57:55.061632: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-11 21:57:55.084200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5114e32858b8): /proc/driver/nvidia/version does not exist
2023-03-11 21:57:55.137982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ExecutionResult(
    component_id: Transform
    execution_id: 42
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [444]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['transformed_metadata', 'metadata', 'transform_fn']

In [445]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [448]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [450]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 4)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'floatList': {'value': [0.5686275]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.9173228]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9212598]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.21902902]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.04886944]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.14263631]}},
                           'Slope_xf': {'int64List': {'value': ['0']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.2214854]}}}}},
 {'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['0']}},
                           'Elevation_xf': {'floatList': {'value': [0.5108095]}},
            

## 5. Metadatos de aprendizaje automático. Seleccion de características

In [454]:
# Import mlmd and utilities
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store

### 5.1. Acceso a artefactos almacenados

In [455]:
# Get the connection config to connect to the context's metadata store
connection_config = context.metadata_connection_config

# Instantiate a MetadataStore instance with the connection config
store = mlmd.MetadataStore(connection_config)

In [456]:
# Get artifact types
artifact_types = store.get_artifact_types()

# Print the results
[artifact_type.name for artifact_type in artifact_types]

['Examples',
 'ExampleStatistics',
 'Schema',
 'ExampleAnomalies',
 'TransformGraph',
 'TransformCache']

### 5.2. Seguimiento de artefactos

In [458]:
# Get artifact types
schema_list = store.get_artifacts_by_type('TransformGraph')

[(f'schema uri: {schema.uri}', f'schema id:{schema.id}') for schema in schema_list]

[('schema uri: ./pipeline/Transform/transform_graph/41', 'schema id:24'),
 ('schema uri: ./pipeline/Transform/transform_graph/42', 'schema id:32')]

In [484]:
# Get artifact types
schema_list = store.get_artifacts_by_type('Schema')

[(f'schema uri: {schema.uri}', f'schema id:{schema.id}') for schema in schema_list]

[('schema uri: ./pipeline/SchemaGen/schema/12', 'schema id:12'),
 ('schema uri: ./pipeline/ImportSchemaGen/schema/14', 'schema id:14'),
 ('schema uri: ./pipeline/ImportSchemaGen/schema/16', 'schema id:15'),
 ('schema uri: ./pipeline/SchemaGen/schema/22', 'schema id:20'),
 ('schema uri: ./pipeline/ImportSchemaGen/schema/23', 'schema id:21'),
 ('schema uri: ./pipeline/Transform/pre_transform_schema/41', 'schema id:27'),
 ('schema uri: ./pipeline/Transform/post_transform_schema/41', 'schema id:29'),
 ('schema uri: ./pipeline/Transform/pre_transform_schema/42', 'schema id:35'),
 ('schema uri: ./pipeline/Transform/post_transform_schema/42', 'schema id:37')]

### 5.3. Obtener artefactos principales

In [485]:
# Get 1st instance of Example
example_artifact = store.get_artifacts_by_type('Schema')[0]

# Print the artifact id
print(f'Artifact id: {example_artifact.id}')

Artifact id: 12


In [486]:
# Get first event related to the ID
example_artifact_id_event = store.get_events_by_artifact_ids([example_artifact.id])[0]

# Print results
print(example_artifact_id_event)

artifact_id: 12
execution_id: 12
path {
  steps {
    key: "schema"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1678568469321



In [487]:
# Get execution ID
example_execution_id = example_artifact_id_event.execution_id

# Get events by the execution ID
events_execution = store.get_events_by_execution_ids([example_execution_id])

# Print results
print(events_execution)

[artifact_id: 11
execution_id: 12
path {
  steps {
    key: "statistics"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1678568469309
, artifact_id: 12
execution_id: 12
path {
  steps {
    key: "schema"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1678568469321
]


In [490]:
# Filter INPUT type events
inputs_to_exval = [event.artifact_id for event in events_execution 
                       if event.type == metadata_store_pb2.Event.INPUT]

# Print results
print(inputs_to_exval)

[11]
